In [1]:
import pandas as pd
import json
import json_lines

In [2]:
### 1. COVID DATA BALI REGENCIES
###  -------------------------------------

## From scrapy spider

In [61]:
## Converting Json-Lines File to pandas DataFrame
##########################################

json_file = r'C:\Users\ansve\Coding\Projects-WebScraping\CovidBali\CovidBali\CovidBali\daily.jl'

##Extract data from First TABLE
###############################################

data_first = []
with open(json_file, 'rb') as f:
    count = 0
    for line in json_lines.reader(f, broken=True):
        count+=1
        if count % 2 == 0: #this is the remainder operator
            for x in range(0, 10):
                data_first.append(line['data'][x])

df_first = pd.DataFrame(data_first)
df_first.to_csv('test.csv')

##Extract data from SECOND TABLE
###############################################
data_second = []

with open(json_file, 'rb') as f:
    count =0
    for line in json_lines.reader(f, broken=True):
        count+= 1
        if count % 2 != 0:
            for x in range(0, 10):
                data_second.append(line['data'][x])
df_second = pd.DataFrame(data_second)
df_second.tail()

df_first.tail()


Error: JSON line can't be decoded. JSONDecodeError("Expecting ',' delimiter: line 1 column 13 (char 12)")
Error: JSON line can't be decoded. JSONDecodeError("Expecting ',' delimiter: line 1 column 13 (char 12)")
Error: JSON line can't be decoded. JSONDecodeError('Extra data: line 1 column 7 (char 6)')
Error: JSON line can't be decoded. JSONDecodeError("Expecting ',' delimiter: line 1 column 13 (char 12)")
Error: JSON line can't be decoded. JSONDecodeError("Expecting ',' delimiter: line 1 column 13 (char 12)")
Error: JSON line can't be decoded. JSONDecodeError('Extra data: line 1 column 7 (char 6)')


,0,1,2,3,4,5,6,7,8,9,10,11,12
625,6,BANGLI,0,58,0,13,18,1548,0,0,18,1619,16 Feb 2021
626,7,KLUNGKUNG,0,19,1,14,12,1183,0,0,13,1216,16 Feb 2021
627,8,KARANGASEM,0,23,0,63,6,1258,0,0,6,1344,16 Feb 2021
628,9,BULELENG,0,41,0,23,32,2064,0,0,32,2128,16 Feb 2021
629,10,KABUPATEN LAINNYA,0,6,2,81,0,235,0,0,2,322,16 Feb 2021


In [60]:

## Merge both df
#############################
df_merged = pd.merge(df_first, df_second, how= 'left', left_on=['1', '12'], right_on=['1', '10'])
df_merged.head()
# df_bali = df_merged.copy()
# df_bali.iloc[10]

,0_x,12,1,2_x,3_x,4_x,5_x,6_x,7_x,8_x,...,3_y,4_y,5_y,6_y,7_y,8_y,9_y,10_y,11_y,12_y
0,1,None,JEMBRANA,-7,70,7,579,1,14,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,None,TABANAN,-10,434,35,1016,0,47,25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,None,BADUNG,-9,159,27,2299,0,51,18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,None,DENPASAR,1,166,26,3705,0,90,27,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,None,GIANYAR,-16,234,31,1681,0,75,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
## Column names from https://infocorona.baliprov.go.id/API/pendataan/laporan-harian-01.php and ...-02.php

columns = [
    'No','Date', 'Kabupaten / County', 'new_treatment', 'total_treatment', 'new_recovered', 'total_recovered', 'new_deaths', 'total_deaths', 'new_cases', 'total_cases', 'Date1', 'no', 'new PPLN/PMI', 'total history-of-foreign-travel/ migrant-worker (PPLN/PMI)', 'new domestic-travel-history', 'total domestic-travel-history (PPDN)', 'new_local_transmission', 'total_local_transmission',	'new_other_transmission', 'total_other_transmission', 'new_cases' ,'total_cases', 'date' ]


In [53]:
# adjust column names

print(len(df_bali.columns), len(columns))


# df_bali.columns= columns
# df_bali.iloc[-1]


23 24


In [36]:
df = df_bali.copy()

In [37]:
# delete duplicate columns and convert dtypes
df = df.loc[:,~df.columns.duplicated()]

## fill NaN in Kabubaten Lain fileds (confirmed cases from outside of bali regency but detected in Bali have no match with reference-data e.g)
################################################
df_merged = df_merged.fillna(0)
df_merged[df_merged.isna().any(axis=1)]

## convert columns
df['No'] = df['No'].astype(int)
df['Name_EN'] = df['Kabupaten / County'].astype(str)
df['Name_EN'] = df['Name_EN'].str.casefold()
df.dtypes

No                                                             int32
Date                                                          object
Kabupaten / County                                            object
new_treatment                                                 object
total_treatment                                               object
new_recovered                                                 object
total_recovered                                               object
new_deaths                                                    object
total_deaths                                                  object
new_cases                                                     object
total_cases                                                   object
Date1                                                         object
no                                                            object
new PPLN/PMI                                                  object
total history-of-foreign-travel/ m

In [38]:
## convert date to datetime 
################################

# from datetime import datetime -> datetime.strptime(date, format)
# df["Date"].apply(lambda x: parse(df['Date']))
# or
# df['Date'] = pd.to_datetime(df['Date'])
## also possible

from dateutil.parser import parse
# check if Nan
# df['Date'].isnull().values

df['Date'] = df['Date'].str.strip()
df['Date'] = df['Date'].astype(str) 
df['Date'] = pd.to_datetime(df.Date, errors='coerce')
df['Date'][0]

Timestamp('2020-12-05 00:00:00')

In [39]:
# data cleaning, processing
##########################

# add addtitional data
df_ref = pd.read_excel('Bali_reference_data.xlsx')
df_ref = df_ref.drop(columns=['https://en.wikipedia.org/wiki/Bali#cite_note-BPS2019-2', 'https://sp2010.bps.go.id/index.php/site/tabel?tid=321&wid=0', 'Source'])

## adjust regency names of both data sources before merge
##########################
df_ref['Name_EN'] = df_ref['Name_Indo'].str.casefold()
df_ref.replace('kota denpasar', 'denpasar', inplace=True)
df_ref.replace('totals', 'bali', inplace=True)
df_ref.replace('Totals', 'Bali', inplace=True)

# df_ref.head(15)

Index([ '0_x',     10,      1,  '2_x',  '3_x',  '4_x',  '5_x',  '6_x',  '7_x',
        '8_x',  '9_x', '10_x', '11_x', '12_x',  '0_y',  '2_y',  '3_y',  '4_y',
        '5_y',  '6_y',  '7_y',  '8_y',  '9_y', '10_y', '11_y', '12_y'],
      dtype='object')

In [40]:
# merge both df on 'Name_EN'
df_merged = df.merge(df_ref, on="Name_EN", how='left')
df_merged.iloc[1]


No                                                                                 2
Date                                                             2020-12-05 00:00:00
Kabupaten / County                                                           TABANAN
new_treatment                                                                    -10
total_treatment                                                                  434
new_recovered                                                                     35
total_recovered                                                                 1016
new_deaths                                                                         0
total_deaths                                                                      47
new_cases                                                                         25
total_cases                                                                     1497
Date1                                                            

In [41]:

df_merged = df_merged.drop(['Date1', 'no'], axis=1)
df_merged.columns

# # convert to integers
int_numbers = ['new_treatment', 'total_treatment',
       'new_recovered', 'total_recovered', 'new_deaths', 'total_deaths',
       'new_cases', 'total_cases', 'new PPLN/PMI',
       'total domestic-travel-history (PPDN)',
       'new domestic-travel-history', 'total domestic-travel-history (PPDN)',
       'new_local_transmission', 'total_local_transmission',
       'new_other_transmission', 'total_other_transmission']
## check for duplicated columns
print (df_merged.columns[df_merged.columns.duplicated(keep=False)])
df_merged[int_numbers] = df_merged[int_numbers].apply(pd.to_numeric)


Index([], dtype='object')


In [42]:


# ## add columns
df_merged['total_cases_per_100k'] = df_merged['total_cases'] / df_merged['population_2015'] * 100000
df_merged['total_cases_per_100k'] = df_merged['total_cases_per_100k'].apply(pd.to_numeric).round(2)

df_merged['total_deaths_per_100k'] = df_merged['total_deaths'] / df_merged['population_2015']
df_merged['total_deaths_per_100k'] = df_merged['total_deaths_per_100k'].apply(pd.to_numeric).round(2)

df_merged['cases7'] = df_merged['new_cases'].rolling(window=7, center=False).mean()
df_merged['cases7_per_100k'] = df_merged['cases7'] / df_merged['population_2015']
df_merged['cases7_per_100k'] = df_merged['cases7_per_100k'].apply(pd.to_numeric).round(2)

df_merged['total_cases_per_100k'] = df_merged['total_cases'] / df_merged['population_2015']
df_merged['total_cases_per_100k'] = df_merged['total_cases_per_100k'].apply(pd.to_numeric).round(2)

df_merged['deaths7'] = df_merged['new_deaths'].rolling(window=7, center=False).mean()
df_merged['deaths7_per_100k'] = df_merged['total_deaths_per_100k'].rolling(window=7, center=False).mean()
df_merged['deaths7_per_100k'] = df_merged['deaths7_per_100k'].apply(pd.to_numeric).round(2)

## Check new created values
##############################
df_merged[['total_deaths', 'total_cases', 'total_deaths_per_100k', 'population_2015', 'total_cases_per_100k']]

,total_deaths,total_cases,total_deaths_per_100k,population_2015,total_cases_per_100k
0,14,663,0.05,271.423,2.44
1,47,1497,0.11,435.753,3.44
2,51,2509,0.08,615.148,4.08
3,90,3961,0.10,879.098,4.51
4,75,1990,0.15,494.729,4.02
...,...,...,...,...,...
625,1548,0,6.96,222.474,0.00
626,1183,0,6.74,175.573,0.00
627,1258,0,3.08,408.487,0.00
628,2064,0,3.20,645.893,0.00


In [43]:

# ## drop temporary unnecessary columns
# drop_columns = ['New PPLN / PMI', 'new PPDN', 'total PPDN',
#        'new_local_transmission', 'total_local_transmission',
#        'new_other_transmission', 'total_other_transmission', 'Capital', 'population_2000',
#        'population_2010', 'HDI_2014_estimate', 'Gouvernor',
#        'Nr. of Sub-Districts', 'Nr of Keluruhan/Villages', 'area_km2' ]
# df_clean = df_merged.drop(columns=drop_columns)

In [44]:
## check NaN in df 
# nan_rows = df_clean[df_clean['population_2015'].isnull()]
# nan_rows

In [45]:
# infection fatality ratio (IFR), which estimates this proportion of deaths among all infected individuals. The second is case fatality ratio (CFR), which estimates this proportion of deaths among identified confirmed cases. 
# see https://www.who.int/news-room/commentaries/detail/estimating-mortality-from-covid-19

df_merged['CFR'] = df_merged['total_deaths'] / df_merged['total_cases'] * 100
df_merged['CFR'] = df_merged['CFR'].round(2)
df_merged['growth_rate_new_cases'] = df_merged['new_cases'].pct_change(fill_method ='ffill', periods=7)

In [46]:
## add id from geojson file
geojson_bali = 'new_bali_id.geojson'

# open new geojson (with id) file
bali_geo_id = json.load(open(geojson_bali, 'r'))

## get names of regencies
regencies = []
for regency in bali_geo_id['features']:
  regencies.append(regency['properties']['ADM2_EN'])

  # add id column in df with feature.id from geojson
bali_id_map = {}
for regency in bali_geo_id['features']:
  bali_id_map[regency['properties']['ADM2_EN']] = regency['id']
bali_id_map

## for Foreigners or Other Regency add 'id' = 0
df_merged['id'] = df_merged['Name_Indo'].apply(lambda x: bali_id_map[x] if x in bali_id_map else 0)
df_merged.head()

,No,Date,Kabupaten / County,new_treatment,total_treatment,new_recovered,total_recovered,new_deaths,total_deaths,new_cases,...,Nr of Keluruhan/Villages,total_cases_per_100k,total_deaths_per_100k,cases7,cases7_per_100k,deaths7,deaths7_per_100k,CFR,growth_rate_new_cases,id
0,1,2020-12-05,JEMBRANA,-7,70,7,579,1,14,1,...,10/41,2.44,0.05,NaN,NaN,NaN,NaN,2.11,NaN,5
1,2,2020-12-05,TABANAN,-10,434,35,1016,0,47,25,...,-/133,3.44,0.11,NaN,NaN,NaN,NaN,3.14,NaN,9
2,3,2020-12-05,BADUNG,-9,159,27,2299,0,51,18,...,16/46,4.08,0.08,NaN,NaN,NaN,NaN,2.03,NaN,1
3,4,2020-12-05,DENPASAR,1,166,26,3705,0,90,27,...,16/27,4.51,0.10,NaN,NaN,NaN,NaN,2.27,NaN,8
4,5,2020-12-05,GIANYAR,-16,234,31,1681,0,75,15,...,6/64,4.02,0.15,NaN,NaN,NaN,NaN,3.77,NaN,4


In [47]:
df_merged.tail()

,No,Date,Kabupaten / County,new_treatment,total_treatment,new_recovered,total_recovered,new_deaths,total_deaths,new_cases,...,Nr of Keluruhan/Villages,total_cases_per_100k,total_deaths_per_100k,cases7,cases7_per_100k,deaths7,deaths7_per_100k,CFR,growth_rate_new_cases,id
625,6,NaT,BANGLI,0,58,0,13,18,1548,0,...,4/68,0.0,6.96,0.0,0.0,29.142857,NaN,inf,NaN,2
626,7,NaT,KLUNGKUNG,0,19,1,14,12,1183,0,...,6/53,0.0,6.74,0.0,0.0,30.714286,7.73,inf,NaN,7
627,8,NaT,KARANGASEM,0,23,0,63,6,1258,0,...,3/75,0.0,3.08,0.0,0.0,31.285714,7.27,inf,NaN,6
628,9,NaT,BULELENG,0,41,0,23,32,2064,0,...,19/129,0.0,3.20,0.0,0.0,33.428571,6.61,inf,NaN,3
629,10,NaT,KABUPATEN LAINNYA,0,6,2,81,0,235,0,...,NaN,NaN,NaN,0.0,NaN,29.142857,NaN,inf,NaN,0


In [48]:
df_merged.to_csv('bali_regency_data.csv')

In [86]:
### 2. INDONESIA COVID DATA 
###  -------------------------------------

## From Kaggel Dataset
# 1. connect to kaggle and get latest dataset


In [87]:
pip install kaggle

Note: you may need to restart the kernel to use updated packages.


In [88]:
from kaggle.api.kaggle_api_extended import KaggleApi

# from keys import kaggle_api

## 1. with Kaggle Api
api =KaggleApi()
api.authenticate()

# downoad single file
#Signature: dataset_download_file(dataset, file_name, path=None, force=False, quiet=True)
api.dataset_download_files('hendratno/covid19-indonesia/covid_19_indonesia_time_series_all.csv', unzip = True)
# 

In [90]:
df_indo = pd.read_csv('covid_19_indonesia_time_series_all.csv')

df_indo.tail()

,Date,Location ISO Code,Location,New Cases,New Deaths,New Recovered,New Active Cases,Total Cases,Total Deaths,Total Recovered,...,Longitude,Latitude,New Cases per Million,Total Cases per Million,New Deaths per Million,Total Deaths per Million,Case Fatality Rate,Case Recovered Rate,Growth Factor of New Cases,Growth Factor of New Deaths
10689,1/21/2021,ID-SG,Sulawesi Tenggara,77,1,1,75,9016,172,7392,...,122.070311,-4.124689,29.22,3421.03,0.38,65.26,1.91%,81.99%,1.97,NaN
10690,1/21/2021,ID-SA,Sulawesi Utara,92,2,83,7,11931,361,8497,...,124.521240,1.259638,34.82,4516.10,0.76,136.64,3.03%,71.22%,0.39,1.00
10691,1/21/2021,ID-SB,Sumatera Barat,181,4,49,128,26091,577,18591,...,100.465062,-0.850253,32.79,4727.28,0.72,104.54,2.21%,71.25%,1.08,0.57
10692,1/21/2021,ID-SS,Sumatera Selatan,101,6,49,46,13437,652,10834,...,104.169465,-3.216212,12.29,1635.16,0.73,79.34,4.85%,80.63%,0.92,6.00
10693,1/21/2021,ID-SU,Sumatera Utara,83,1,80,2,19962,719,17228,...,99.051964,2.191894,5.58,1341.99,0.07,48.34,3.60%,86.30%,1.04,0.50


In [91]:
## convert dates to datetime format so matching with new dataframe before merging

from datetime import datetime
print(df_indo['Date'][0])
df_indo['Date'] = pd.to_datetime(df_indo['Date'])

## with datetime libary
# df_indo['Date'] = datetime.strptime(df_indo['Date'], '%m/%d/%Y').date()
# datetime.strptime(df_indo["Date"][0], '%m/%d/%Y').date()

df_indo['Date'][0]

3/1/2020


Timestamp('2020-03-01 00:00:00')

In [92]:
df_indo.head()

,Date,Location ISO Code,Location,New Cases,New Deaths,New Recovered,New Active Cases,Total Cases,Total Deaths,Total Recovered,...,Longitude,Latitude,New Cases per Million,Total Cases per Million,New Deaths per Million,Total Deaths per Million,Case Fatality Rate,Case Recovered Rate,Growth Factor of New Cases,Growth Factor of New Deaths
0,2020-03-01,ID-JK,DKI Jakarta,2,0,0,2,489,20,39,...,106.836118,-6.204699,0.18,45.09,0.0,1.84,4.09%,7.98%,NaN,NaN
1,2020-03-02,ID-JK,DKI Jakarta,2,0,0,2,491,20,39,...,106.836118,-6.204699,0.18,45.27,0.0,1.84,4.07%,7.94%,1.0,1.0
2,2020-03-02,IDN,Indonesia,2,0,0,2,2,0,0,...,113.921327,-0.789275,0.01,0.01,0.0,0.00,0.00%,0.00%,NaN,NaN
3,2020-03-02,ID-JB,Jawa Barat,3,0,0,3,12,5,135,...,107.603708,-6.920432,0.07,0.27,0.0,0.11,41.67%,1125.00%,NaN,NaN
4,2020-03-02,ID-RI,Riau,1,0,0,1,2,1,1,...,101.805109,0.511648,0.16,0.33,0.0,0.16,50.00%,50.00%,NaN,NaN


In [93]:
## Get Bali and Indo Data from new Xlsx Sheet
## Source Kawal-Indo

df_indo_new = pd.read_excel('Indo_data.xlsx', 'Bali and Indo')
df_indo_new= df_indo_new.sort_values(by=['Date'], ascending=True)
df_indo_new['Date'][0]

## Merge with old data df

df_indo = df_indo.append(df_indo_new)

# check values
df_indo.head()


,Date,Location ISO Code,Location,New Cases,New Deaths,New Recovered,New Active Cases,Total Cases,Total Deaths,Total Recovered,...,Latitude,New Cases per Million,Total Cases per Million,New Deaths per Million,Total Deaths per Million,Case Fatality Rate,Case Recovered Rate,Growth Factor of New Cases,Growth Factor of New Deaths,Total Active Cases
0,2020-03-01,ID-JK,DKI Jakarta,2,0,0,2.0,489,20,39,...,-6.204699,0.18,45.09,0.0,1.84,4.09%,7.98%,NaN,NaN,NaN
1,2020-03-02,ID-JK,DKI Jakarta,2,0,0,2.0,491,20,39,...,-6.204699,0.18,45.27,0.0,1.84,4.07%,7.94%,1.0,1.0,NaN
2,2020-03-02,IDN,Indonesia,2,0,0,2.0,2,0,0,...,-0.789275,0.01,0.01,0.0,0.00,0.00%,0.00%,NaN,NaN,NaN
3,2020-03-02,ID-JB,Jawa Barat,3,0,0,3.0,12,5,135,...,-6.920432,0.07,0.27,0.0,0.11,41.67%,1125.00%,NaN,NaN,NaN
4,2020-03-02,ID-RI,Riau,1,0,0,1.0,2,1,1,...,0.511648,0.16,0.33,0.0,0.16,50.00%,50.00%,NaN,NaN,NaN


In [94]:
# df_indo.dtypes

In [95]:
print(list(df_merged.columns))

['No', 'Date', 'Kabupaten / County', 'new_treatment', 'total_treatment', 'new_recovered', 'total_recovered', 'new_deaths', 'total_deaths', 'new_cases', 'total_cases', 'new PPLN/PMI', 'total history-of-foreign-travel/ migrant-worker (PPLN/PMI)', 'new domestic-travel-history', 'total domestic-travel-history (PPDN)', 'new_local_transmission', 'total_local_transmission', 'new_other_transmission', 'total_other_transmission', 'date', 'Name_EN', 'Name_Indo', 'Capital', 'area_km2', 'population_2000', 'population_2010', 'population_2015', 'HDI_2014_estimate', 'Gouvernor', 'Nr. of Sub-Districts', 'Nr of Keluruhan/Villages', 'total_cases_per_100k', 'total_deaths_per_100k', 'cases7', 'cases7_per_100k', 'deaths7', 'deaths7_per_100k', 'CFR', 'growth_rate_new_cases', 'id']


In [96]:
df_indo.columns

Index(['Date', 'Location ISO Code', 'Location', 'New Cases', 'New Deaths',
       'New Recovered', 'New Active Cases', 'Total Cases', 'Total Deaths',
       'Total Recovered', 'Total Active Cases', 'Location Level',
       'City or Regency', 'Province', 'Country', 'Continent', 'Island',
       'Time Zone', 'Special Status', 'Total Regencies', 'Total Cities',
       'Total Districts', 'Total Urban Villages', 'Total Rural Villages',
       'Area (km2)', 'Population', 'Population Density', 'Longitude',
       'Latitude', 'New Cases per Million', 'Total Cases per Million',
       'New Deaths per Million', 'Total Deaths per Million',
       'Case Fatality Rate', 'Case Recovered Rate',
       'Growth Factor of New Cases', 'Growth Factor of New Deaths',
       'Total Active Cases'],
      dtype='object')

In [97]:
# adjust column names
columns_new = ['Date', 'Location ISO Code', 'Location', 'new_cases', 'new_deaths',
       'new_recovered', 'New Active Cases', 'total_cases', 'total_deaths',
       'total_recovered', 'Total Active Cases', 'Location Level',
       'City or Regency', 'Province', 'Country', 'Continent', 'Island',
       'Time Zone', 'Special Status', 'Total Regencies', 'Total Cities',
       'Total Districts', 'Total Urban Villages', 'Total Rural Villages',
       'Area (km2)', 'Population', 'Population Density', 'Longitude',
       'Latitude', 'New Cases per Million', 'Total Cases per Million',
       'New Deaths per Million', 'Total Deaths per Million',
       'CFR', 'Case Recovered Rate',
       'Growth Factor of New Cases', 'Growth Factor of New Deaths', 'Total Active Cases']

df_indo.columns = columns_new
df_indo.columns

Index(['Date', 'Location ISO Code', 'Location', 'new_cases', 'new_deaths',
       'new_recovered', 'New Active Cases', 'total_cases', 'total_deaths',
       'total_recovered', 'Total Active Cases', 'Location Level',
       'City or Regency', 'Province', 'Country', 'Continent', 'Island',
       'Time Zone', 'Special Status', 'Total Regencies', 'Total Cities',
       'Total Districts', 'Total Urban Villages', 'Total Rural Villages',
       'Area (km2)', 'Population', 'Population Density', 'Longitude',
       'Latitude', 'New Cases per Million', 'Total Cases per Million',
       'New Deaths per Million', 'Total Deaths per Million', 'CFR',
       'Case Recovered Rate', 'Growth Factor of New Cases',
       'Growth Factor of New Deaths', 'Total Active Cases'],
      dtype='object')

In [98]:
# add columns so consistent with Bali_regency data
df_indo['Name_EN'] = df_indo['Location'].str.casefold()

df_indo['new_cases_per_100k'] = df_indo['new_cases'] / df_indo['Population'] * 100000.0
df_indo['new_cases_per_100k'] = df_indo['new_cases_per_100k'].round(2)

df_indo['new_deaths_per_100k'] = df_indo['new_deaths'] / df_indo['Population'] * 100000.0
df_indo['new_deaths_per_100k']= df_indo['new_deaths_per_100k'].round(2)

df_indo['total_cases_per_100k'] = df_indo['total_cases'] / df_indo['Population'] * 100000.0
df_indo['total_cases_per_100k'] = df_indo['total_cases_per_100k'].round(2)

df_indo['total_deaths_per_100k'] = df_indo['total_deaths'] / df_indo['Population'] * 100000.0
df_indo['total_deaths_per_100k'] = df_indo['total_deaths_per_100k'].round(2)

# rolling 7-day average
df_indo['cases7'] = df_indo['new_cases'].rolling(window=7, center=False).mean().round(2)
df_indo['deaths7'] = df_indo['new_deaths'].rolling(window=7, center = False).mean().round(2)

df_indo['cases7_per_100k'] = df_indo['cases7'] / df_indo['Population'] *100000
df_indo['deaths7_per_100k'] = df_indo['deaths7'] / df_indo['Population'] *100000
df_indo['cases7_per_100k'] = df_indo['cases7_per_100k'].round(2)
df_indo['deaths7_per_100k'] = df_indo['deaths7_per_100k'].round(2)

# df_indo['growth_rate'] = df_indo['Growth Factor of New Cases']
df_indo['growth_rate_new_cases'] = df_indo['new_cases'].pct_change(periods=7, fill_method='ffill')

df_indo.tail()

,Date,Location ISO Code,Location,new_cases,new_deaths,new_recovered,New Active Cases,total_cases,total_deaths,total_recovered,...,Name_EN,new_cases_per_100k,new_deaths_per_100k,total_cases_per_100k,total_deaths_per_100k,cases7,deaths7,cases7_per_100k,deaths7_per_100k,growth_rate_new_cases
15,2021-01-16,NaN,Bali,319,3,149,NaN,21030,586,18217,...,bali,7.57,0.07,498.79,13.90,7257.71,161.86,172.14,3.84,0.190299
34,2021-01-17,NaN,Indonesia,11287,220,9102,NaN,907929,25987,736460,...,indonesia,4.26,0.08,342.38,9.80,7259.00,149.57,2.74,0.06,0.000798
16,2021-01-17,NaN,Bali,262,1,129,NaN,21292,587,18346,...,bali,6.21,0.02,505.01,13.92,5645.43,107.57,133.90,2.55,-0.977330
17,2021-01-18,NaN,Bali,238,4,251,NaN,21530,591,18597,...,bali,5.64,0.09,510.65,14.02,5637.00,107.86,133.70,2.56,-0.198653
35,2021-01-18,NaN,Indonesia,9086,295,9475,NaN,917015,26282,745935,...,indonesia,3.43,0.11,345.80,9.91,5103.86,116.00,1.92,0.04,-0.291153


In [99]:
## add id from geojson for choropleth Map
#########################################

# open new geojson (with id) file
geojson_indo = 'new_indo_id.geojson'
indo_geo_id = json.load(open(geojson_indo, 'r'))


In [100]:
indo_geo_id['features'][7]['id']

8

In [101]:
## ADJUST NAMES OF df_indo with NAMES OF GEOJSON ADM1_EN

# which names are different
names_df = df_indo['Location'].unique()
provinces = []
for province in indo_geo_id['features']:
  provinces.append(province['properties']['state'])

list(set(provinces) - set(names_df))

['Jakarta Raya', 'Irian Jaya Barat', 'Bangka-Belitung', 'Yogyakarta']

In [102]:
list_df = ['DKI Jakarta', 'Kalimantan Utara', 'Daerah Istimewa Yogyakarta', 'Papua Barat','Kepulauan Bangka Belitung']

list(set(provinces) - set(names_df))
list1= ['Bangka-Belitung', 'Jakarta Raya', 'Yogyakarta', 'Irian Jaya Barat']


In [103]:
# adjust Names of provnices in Indo

df_indo = df_indo.replace("DKI Jakarta", 'Jakarta Raya')
df_indo = df_indo.replace("Papua Barat", 'Irian Jaya Barat')
df_indo = df_indo.replace("Daerah Istimewa Yogyakarta", 'Yogyakarta')
df_indo = df_indo.replace("Kepulauan Bangka Belitung", 'Bangka-Belitung')

In [104]:
provinces = []
for province in indo_geo_id['features']:
  provinces.append(province['properties']['state'])

# add id column in df with feature.id from geojson
indo_id_map = {}
for province in indo_geo_id['features']:
  indo_id_map[province['properties']['state']] = province['id']
# indo_id_map

# ## for Foreigners or Other Regency add 'id' = 0
df_indo['id'] = df_indo['Location'].apply(lambda x: indo_id_map[x] if x in indo_id_map else 0)

df_indo.tail(40)


,Date,Location ISO Code,Location,new_cases,new_deaths,new_recovered,New Active Cases,total_cases,total_deaths,total_recovered,...,new_cases_per_100k,new_deaths_per_100k,total_cases_per_100k,total_deaths_per_100k,cases7,deaths7,cases7_per_100k,deaths7_per_100k,growth_rate_new_cases,id
10690,2021-01-21,ID-SA,Sulawesi Utara,92,2,83,7.0,11931,361,8497,...,3.48,0.08,451.61,13.66,188.57,2.29,7.14,0.09,0.022222,22
10691,2021-01-21,ID-SB,Sumatera Barat,181,4,49,128.0,26091,577,18591,...,3.28,0.07,472.73,10.45,207.57,2.71,3.76,0.05,2.770833,2
10692,2021-01-21,ID-SS,Sumatera Selatan,101,6,49,46.0,13437,652,10834,...,1.23,0.07,163.52,7.93,202.43,3.00,2.46,0.04,-0.262774,33
10693,2021-01-21,ID-SU,Sumatera Utara,83,1,80,2.0,19962,719,17228,...,0.56,0.01,134.20,4.83,179.86,2.57,1.21,0.02,-0.655602,4
0,2021-01-01,NaN,Bali,101,1,93,NaN,17694,520,16124,...,2.40,0.02,419.67,12.33,102.14,2.43,2.42,0.06,-0.843411,24
18,2021-01-01,NaN,Indonesia,8072,191,6839,NaN,751270,22329,617936,...,3.04,0.07,283.30,8.42,1243.86,29.43,0.47,0.01,99.900000,0
1,2021-01-02,NaN,Bali,165,5,128,NaN,17859,525,16252,...,3.91,0.12,423.58,12.45,1256.43,30.00,29.80,0.71,1.142857,24
19,2021-01-02,NaN,Indonesia,7203,226,7582,NaN,758473,22555,625518,...,2.72,0.09,286.02,8.51,2272.29,62.00,0.86,0.02,77.293478,0
2,2021-01-03,NaN,Bali,119,2,78,NaN,17978,527,16330,...,2.82,0.05,426.41,12.50,2263.43,61.71,53.68,1.46,-0.342541,24
20,2021-01-03,NaN,Indonesia,6877,179,6419,NaN,765350,22734,631937,...,2.59,0.07,288.61,8.57,3231.43,86.43,1.22,0.03,67.089109,0


In [105]:
df_indo.to_csv('indo_province_data.csv')

In [35]:
### 3. World Data
##------------------------


from kaggle.api.kaggle_api_extended import KaggleApi

# from keys import kaggle_api

## 1. with Kaggle Api
api =KaggleApi()
api.authenticate()

# downoad single file
#Signature: dataset_download_file(dataset, file_name, path=None, force=False, quiet=True)
# api.dataset_download_files('imdevskp/corona-virus-report/worldometer_data.csv', unzip=True)

# ## Datasets

# REALLY GOOD !!!!
# 
https://github.com/owid/covid-19-data/blob/master/public/data/owid-covid-data.csv
